In [1]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import astropy.units as u
import astropy.constants as const
%matplotlib inline

In [13]:
G = const.G
pi = np.pi
msun = u.Msun
rsun = u.Rsun
kms = (u.km) / (u.s)

In [5]:
def kick(vkick_min, vkick_max, N):
    """
    Generates kick magnitudes and directions.

    Parameters:
        vkick_min (float): Minimum kick magnitude.
        vkick_max (float): Maximum kick magnitude.
        N (int): Number of samples to generate.

    Returns:
        tuple: 
            - magnitudes (array): Kick magnitudes (size N).
            - directions (array): Array of polar and azimuthal angles (size N x 2).
    """
    # Generate magnitudes uniformly
    magnitudes = np.random.uniform(vkick_min, vkick_max, N)
    
    # Generate isotropic directions
    theta = np.arccos(2 * np.random.random(N) - 1)  # Polar angle
    phi = 2 * np.pi * np.random.random(N)          # Azimuthal angle
    
    # Combine directions
    directions = np.vstack((theta, phi)).T
    
    return magnitudes, directions

def sample_orbital_period(P_min, P_max, N):
    """
    Generates a uniform distribution of orbital periods.

    Parameters:
        P_min (float): Minimum orbital period (days).
        P_max (float): Maximum orbital period (days).
        N (int): Number of samples to generate.

    Returns:
        np.ndarray: Array of sampled orbital periods (size N).
    """
    return np.random.uniform(P_min, P_max, N)

def sample_mass_loss(delta_m_min, delta_m_max, N):
    """
    Generates a uniform distribution of mass loss values during the SN.

    Parameters:
        delta_m_min (float): Minimum mass loss (M_sun).
        delta_m_max (float): Maximum mass loss (M_sun).
        N (int): Number of samples to generate.

    Returns:
        np.ndarray: Array of sampled mass loss values (size N).
    """
    return np.random.uniform(delta_m_min, delta_m_max, N)

def calculate_progenitor_mass(delta_m, M_min):
    """
    Calculates the progenitor mass based on mass loss.

    Parameters:
        delta_m (np.ndarray): Array of mass loss values (M_sun).
        M_min (float): Minimum progenitor mass (M_sun), corresponding to zero mass loss.

    Returns:
        np.ndarray: Array of progenitor masses (size N).
    """
    return M_min + delta_m

def sample_eccentricities(N):
    """
    Generates a uniform distribution of eccentricities.

    Parameters:
        N (int): Number of samples to generate.

    Returns:
        np.ndarray: Array of sampled eccentricities (size N) between 0 and 1.
    """
    return np.random.uniform(0, 1, N)

def sample_mean_anomaly(N):
    """
    Generates a uniform distribution of mean anomalies.

    Parameters:
        N (int): Number of samples to generate.

    Returns:
        np.ndarray: Array of sampled mean anomalies (size N) between 0 and 2*pi radians.
    """
    return np.random.uniform(0, 2 * np.pi, N)


In [11]:
def v_orb(M1, M2, a):
    """
    Computes the post-SN eccentricity.

    Parameters:
        m_tilde (float): Ratio of pre- to post-SN total masses.
        v_tilde (float): Scaled kick velocity (v_kick / v_orb).
        phi (float): Angle between orbital and kick velocity vectors (radians).
        theta (float): Angle between orbital plane and kick velocity vector (radians).

    Returns:
        float: Post-SN eccentricity.
    """
    return (np.sqrt(G * (M1 + M2) / a)).to('km/s')

def P_orb(M1, M2, a):
    """
    Computes the post-SN eccentricity.

    Parameters:
        m_tilde (float): Ratio of pre- to post-SN total masses.
        v_tilde (float): Scaled kick velocity (v_kick / v_orb).
        phi (float): Angle between orbital and kick velocity vectors (radians).
        theta (float): Angle between orbital plane and kick velocity vector (radians).

    Returns:
        float: Post-SN eccentricity.
    """
    return (2 * pi * np.sqrt(a**3 / (G * (M1 + M2)))).to('day')

def m_tilde(M1, M1_prime, M2):
    """
    Computes the post-SN eccentricity.

    Parameters:
        m_tilde (float): Ratio of pre- to post-SN total masses.
        v_tilde (float): Scaled kick velocity (v_kick / v_orb).
        phi (float): Angle between orbital and kick velocity vectors (radians).
        theta (float): Angle between orbital plane and kick velocity vector (radians).

    Returns:
        float: Post-SN eccentricity.
    """
    return (M1 + M2) / (M1_prime + M2)

def v_tilde(v_kick, v_orb):
    """
    Computes the post-SN eccentricity.

    Parameters:
        m_tilde (float): Ratio of pre- to post-SN total masses.
        v_tilde (float): Scaled kick velocity (v_kick / v_orb).
        phi (float): Angle between orbital and kick velocity vectors (radians).
        theta (float): Angle between orbital plane and kick velocity vector (radians).

    Returns:
        float: Post-SN eccentricity.
    """
    return v_kick / v_orb

def post_SN_eccentricity(m_tilde, v_tilde, phi, theta):
    """
    Computes the post-SN eccentricity.

    Parameters:
        m_tilde (float): Ratio of pre- to post-SN total masses.
        v_tilde (float): Scaled kick velocity (v_kick / v_orb).
        phi (float): Angle between orbital and kick velocity vectors (radians).
        theta (float): Angle between orbital plane and kick velocity vector (radians).

    Returns:
        float: Post-SN eccentricity.
    """
    term1 = 2 - m_tilde * (1 + 2 * v_tilde * np.cos(phi) * np.cos(theta) + v_tilde**2)
    term2 = (1 + v_tilde * np.cos(phi) * np.cos(theta))**2 + (v_tilde * np.sin(theta))**2
    return np.sqrt(1 - m_tilde * term1 * term2)

def post_SN_semi_major_axis(a, m_tilde, v_tilde, phi, theta):
    """
    Computes the post-SN semi-major axis.

    Parameters:
        a (float): Pre-SN semi-major axis (m).
        m_tilde (float): Ratio of pre- to post-SN total masses.
        v_tilde (float): Scaled kick velocity (v_kick / v_orb).
        phi (float): Angle between orbital and kick velocity vectors (radians).
        theta (float): Angle between orbital plane and kick velocity vector (radians).

    Returns:
        float: Post-SN semi-major axis (m).
    """
    term1 = 2 - m_tilde * (1 + 2 * v_tilde * np.cos(phi) * np.cos(theta) + v_tilde**2)
    return a / term1

def post_SN_system_velocity(M1, M1_prime, M2, v_orb, v_tilde, phi, theta):
    """
    Computes the post-SN systemic velocity of the binary system.

    Parameters:
        M1 (float): Pre-SN mass of the primary star (kg).
        M1_prime (float): Post-SN mass of the primary star (kg).
        M2 (float): Mass of the secondary star (kg).
        v_orb (float): Initial orbital velocity (m/s).
        v_tilde (float): Scaled kick velocity (v_kick / v_orb).
        phi (float): Angle between orbital and kick velocity vectors (radians).
        theta (float): Angle between orbital plane and kick velocity vector (radians).

    Returns:
        float: Post-SN systemic velocity (m/s).
    """
    # Calculate mass ratio μ and mass loss ΔM1
    mu = (M1 * M2) / (M1 + M2)  # Reduced mass
    delta_M1 = M1 - M1_prime  # Mass lost during SN
    
    # Calculate the systemic velocity
    term1 = (mu * delta_M1 / M1)**2
    term2 = -2 * (mu * delta_M1 * M1_prime / M1) * v_tilde * np.cos(phi) * np.cos(theta)
    term3 = (M1_prime * v_tilde)**2
    v_sys = (v_orb / (M1_prime + M2)) * np.sqrt(term1 + term2 + term3)
    
    return v_sys

In [12]:
M1 = 60 * msun
M1_prime = 33 * msun
M2 = 0.8 * msun
a = 3000 * rsun
x,y = kick(0, 100, 1)
v_kick = x[0][0] * kms


vorb = v_orb(M1, M2, a)
prob = P_orb(M1, M2, a)
mtilde = m_tilde(M1, M1_prime, M2)
vtilde = v_tilde(v_kick, v_orb)
e = post_SN_eccentricity(m_tilde, v_tilde, phi, theta)

(<Quantity 62.17788963 km / s>, <Quantity 2441.031627 d>)

In [24]:
kick(0, 50, 1)[1][0,0]

np.float64(2.315877048449753)